# LGBIO2110 - Introduction to clinical engineering
## Project : Epidemiology modelling of COVID-19

__Date :__ Year 2021-2022

__Professor :__ Philippe Lefèvre and Benoît Delhaye

__Author :__ Benoît Delhaye and Donatien Doumont

__Content :__ At the end of this project, you should master and understand the following :


*   Master and be able to use mathematical modelling of a epidemy
*   Understanding the mechanisms that influence the disease spread and their dynamics. Be able to relate the dynamical evolution of the epidemy to events (political decisions, appearance of virus variant, vaccination,...) 
*   Predict the future trend : how severe will the pandemic be ? 
*   Suggest control strategies and evaluate their effects


In [27]:
#Librairies utiles 
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
import numpy as np
import urllib3.request
import pandas as pd
import ipywidgets as widgets
%config InlineBackend.figure_format = 'retina'

# use NMA plot style
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")
my_layout = widgets.Layout()

<font size=5 color=#009999> Context  </font> <br> <br>
SIR model from Kermack-McKendrick is expressed using only three *communicating vessel*
*   Susceptible S(t)
*   Infected I(t)
*   Removed R(t) caused by isolation/recovery/death

Dynamical equations of the model are the following : 
\begin{align*}
    \dot{S} &= -\beta S I\\
    \dot{I} &= \beta S I - \alpha I\\
    \dot{R} &= \alpha I\\
\end{align*}
Remark that the hypothesis of a constant population is made here, hence $\dot{S} + \dot{I} + \dot{R} = 0$ is verified.

***
Example : Behavior of the model for a given initial value for the susceptible $S(t)$ and infected $I(t)$ group, and for constant beta $\beta$ and alpha $\alpha$ parameters.
***
Play with the widgets parameters to better interpret their impact on the model.

In [28]:
#SIR model discrete computation 
def SIRmodel(S0,I0,param):
    beta = param[0]
    alpha = param[1]
    dt = param[2]
    #Initialisation
    S = np.zeros((N,1))
    I = S.copy()
    R = S.copy() 
    S[0] = S0 
    I[0] = I0
    for i in range(0,N-1) : 
        #print(i)
        S[i+1] = S[i]+(-beta*S[i]*I[i])*dt
        I[i+1] = I[i]+(beta*S[i]*I[i]-alpha*I[i])*dt
        R[i+1] = R[i]+(alpha*I[i])*dt
    return S, I, R

#Graphical representation
def plot_SIRmodel(t_vector, S, I, R):
    plt.plot(t_vector, S, label = 'S(t)')
    plt.plot(t_vector, I, label = 'I(t)')
    plt.plot(t_vector, R, label = 'R(t)')
    plt.legend()
    plt.xlabel("Time [days]")
    plt.ylabel("Relative group size")
    plt.xlim((0,t_vector[-1]))
    plt.ylim((0,1))
    

In [29]:
#--------------------------------------------------
tend = 60 #in days
Fs = 1e2 #sample frequency
N = int(tend*Fs) #sample size
time = np.linspace(0,tend,N) #time sequence

#Initial condition of the epidemic
S0 = 0.9
I0 = 0.1

def refresh(beta=0.5,alpha_over_one=10):
    #Dynamical evolution of the model
    alpha = 1/alpha_over_one
    Repro0 = beta*S0/alpha
    S,I,R = SIRmodel(S0,I0,[beta, alpha, 1/Fs])
    plot_SIRmodel(time, S, I, R)
    plt.title("$R_0$ = " + str(Repro0))
    plt.show()
style = {'description_width' : 'initial'}

_ = widgets.interact(refresh,
    beta = widgets.FloatLogSlider(value=0.5, min=-2, max=1, steps=0.05, description="Logarithmic slider: beta", style = style),
    alpha_over_one = widgets.IntSlider(value=10, min=1, max=30, step=1, description="Linear slider: 1/alpha(in days):",style=style),
)

interactive(children=(FloatLogSlider(value=0.5, description='Logarithmic slider: beta', max=1.0, min=-2.0, sty…

<font size=5 color=#009999> Homework  </font> <br> <br>
Instructions : 
1. Collect Belgian data 
2. Adjust the infected population number (being underestimated in the collected dataset)
3. Estimate the time varying reproduction number $R_t$ with an appropiate simple model and show the effect of government measures on the time variyng $R_t$ 
4. Forecast the evolution of $\beta(t)$ and predict the evolution of the pandemic 
5. Include the effect of vaccine 


The dataset used in this project is restricted to the following time period : from $t_0$ = **March, 2020** to $t_1$ = **March, 2021**. 

You will have to predict the behaviour of the epidemy at least **three months** after $t_1$. 

***
1) Importation of dataset
***
Open accessed data are imported from webpage sciensano and are preprocessed. 

Rem : The dataset is created in csv file in a folder data

In [30]:
## Import Data from sciensano web page
# epidemiology status for : 
# *     Hospitalisation 
# *     Death
# *     Age and Sex 
# *     Tests
# *     Vaccination

# Debug console gives for example :  
# https://epistat.sciensano.be/Data/COVID19BE_CASES_AGESEX.csv

weblink = "https://epistat.sciensano.be/Data/"
fn = ["HOSP", "MORT", "CASES_AGESEX", "tests", "VACC"]

for i in range(len(fn)):
    fnloc = "../data/COVID19BE_" + fn[i] + ".csv"
    try:
        with urllib.request.urlopen(weblink+"COVID19BE_"+fn[i]+".csv") as f:
            mycontent = f.read().decode('utf-8')
    except urllib.error.URLError as e:
        print(e.reason)
    fileID = open(fnloc, "w", encoding="utf-8")
    fileID.write(mycontent)
    fileID.close()
    
    #Creation of a table 
    df = pd.read_csv(fnloc, sep=",", header='infer', parse_dates=["DATE"])
    #df["DATE"] = pd.to_datetime(df["DATE"], format='%d-%m-%y', unit='D')
    
    # Beware that some region are NaN
    # boolregionnull = df.REGION.isnull()
    
    # Add new variable to dataframe to have a count of each dose of vaccin
    # DOSEA = First dose for Pfizer-BioNTech, Moderna and AstraZeneca-Oxford
    # DOSEB = Second dose for Pfizer-BioNTech, Moderna and AstraZeneca-Oxford
    # DOSEC = First dose for Johnson&Johnson 
    # DOSED = *** non existant ***
    # DOSEE = Third dose for Pfizer-BioNTech, Moderna and AstraZeneca-Oxford
    # DOSEF = *** non existant ***
    if fn[i] == "VACC": 
        df.loc[:,"DOSEA"] = (df.DOSE=="A")*df.COUNT
        df.loc[:,"DOSEB"] = (df.DOSE=="B")*df.COUNT
        df.loc[:,"DOSEC"] = (df.DOSE=="C")*df.COUNT
        df.loc[:,"DOSEE"] = (df.DOSE=="E")*df.COUNT
    
    #Keep only numerical quantities in the dataset
    fieldn = df.columns
    dfnum = df.select_dtypes(include='number')
    vars = dfnum.columns
    dfnum.insert(0,'DATE', df.DATE, True)
    
    #Sum values over the same day
    dfsum = dfnum.groupby(['DATE'], as_index = True).sum()
    for ii in range(len(vars)):
        exec(fn[i] +'_' +vars[ii]+' = dfsum.'+vars[ii])
    exec(fn[i] +'_DATE = df.DATE.unique()')
    
    #Print of the head of the file (to comment)
    #print(df.head())

[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)


FileNotFoundError: [Errno 2] No such file or directory: '../data/COVID19BE_HOSP.csv'

Variable available across time :
- from HOSP :           HOSP_DATE, HOSP_NEW_IN, HOSP_NEW_OUT, HOSP_NR_REPORTING, HOSP_TOTAL_IN, HOSP_TOTAL_IN_ECMO, HOSP_TOTAL_IN_ICU, HOSP_TOTAL_IN_RESP.
- from MORT :           MORT_DATE, MORT_DEATHS.
- from CASES_AGESEX :   CASES_AGESEX_DATE, CASES_AGESEX_CASES.
- from tests :          tests_DATE, tests_TESTS_ALL, tests_TESTS_ALL_POS.
- from VACC :           VACC_DATE, VACC_COUNT, VACC_DOSEA, VACC_DOSEB, VACC_DOSEC, VACC_DOSEE.

Beware that each 
***
2. Infected population estimation : The goal is to better estimate the infected population begin underestimated in the dataset.  
***

In [5]:
## I(t) estimation 

######################
### Your code here ###
######################

***
3) Fit Data : The goal here is to fit the beta coefficient $\beta(t)$ with the data across time.
You will need to improve your model from the basic SIR model to a SEIR model that account for exposed population group. This group is infected but not contagious for a period of time. 
***

In [6]:
## Fit Data

######################
### Your code here ###
######################

***
4) Predict the future : The goal is to make prediction about the evolution of beta after $t_1$
*** 

In [7]:
## Predict the future based on your model

######################
### Your code here ###
######################

***
5) Improve the model to take into account additional features.
***

In [8]:
## Improvements 

######################
### Your code here ###
######################